In [1]:
from typing import List, Tuple

In [2]:
# Test
file_name="./aoc_2116_test.txt"
# Prod
file_name="./aoc_2116_prod.txt"

In [3]:
with open(file_name) as f:
    raw_data=[line for line in f.read().splitlines()]
    f.close()

# Check
# display(raw_data[:10])

In [4]:
data=raw_data[0]
data

'6053231004C12DC26D00526BEE728D2C013AC7795ACA756F93B524D8000AAC8FF80B3A7A4016F6802D35C7C94C8AC97AD81D30024C00D1003C80AD050029C00E20240580853401E98C00D50038400D401518C00C7003880376300290023000060D800D09B9D03E7F546930052C016000422234208CC000854778CF0EA7C9C802ACE005FE4EBE1B99EA4C8A2A804D26730E25AA8B23CBDE7C855808057C9C87718DFEED9A008880391520BC280004260C44C8E460086802600087C548430A4401B8C91AE3749CF9CEFF0A8C0041498F180532A9728813A012261367931FF43E9040191F002A539D7A9CEBFCF7B3DE36CA56BC506005EE6393A0ACAA990030B3E29348734BC200D980390960BC723007614C618DC600D4268AD168C0268ED2CB72E09341040181D802B285937A739ACCEFFE9F4B6D30802DC94803D80292B5389DFEB2A440081CE0FCE951005AD800D04BF26B32FC9AFCF8D280592D65B9CE67DCEF20C530E13B7F67F8FB140D200E6673BA45C0086262FBB084F5BF381918017221E402474EF86280333100622FC37844200DC6A8950650005C8273133A300465A7AEC08B00103925392575007E63310592EA747830052801C99C9CB215397F3ACF97CFE41C802DBD004244C67B189E3BC4584E2013C1F91B0BCD60AA1690060360094F6A70B7FC7D34A52CBAE011CB6A17509F8

## Part 0

In [5]:
str(bin(int('0',16)))[2:].zfill(4)
int('011111100101',2)

2021

In [6]:
def hex2bin(msg_hex: str) -> str:
    return ''.join([str(bin(int(hex_digit,16)))[2:].zfill(4) for hex_digit in msg_hex])
#hex2bin(data)

In [7]:
def decode_value(encoded_value_bin: str) -> Tuple[int, str]:
    value_bin=''
    bit_position=0
    while encoded_value_bin[bit_position]!='0':
        value_bin+=encoded_value_bin[bit_position+1:bit_position+5]
        bit_position+=5
    value_bin+=encoded_value_bin[bit_position+1:bit_position+5]
    bit_position+=5
    value_dec=int(value_bin, 2)
    return (value_dec, encoded_value_bin[bit_position:])

In [8]:
decode_value('101111111000101000')

(2021, '000')

## Part 1

In [9]:
def parse_msg(pack_bin: str, nr_of_packs=1) -> List[Tuple[int, int, int, str, int]]:
    sub_packs_tuples=[]
    my_calc_value=0
    #Header
    my_version_bin=pack_bin[:3]
    my_version_dec=int(my_version_bin, 2)
    my_type_id_bin=pack_bin[3:6]
    my_type_id_dec=int(my_type_id_bin, 2)
    
    #Value or package
    if my_type_id_dec==4:
        # Value
        my_calc_value, my_remain_part = decode_value(pack_bin[6:])
        # print(f"Extracted value {my_calc_value}, remained {my_remain_part}")
        remaining_packs=nr_of_packs-1
    else:
        #Operator
        pack_len_type_id=int(pack_bin[6])
        # print(pack_len_type_id)
        if pack_len_type_id==0:
            #Fixed bits lengths
            sub_packs_len_bin=pack_bin[7:7+15]
            sub_packs_len_dec=int(sub_packs_len_bin, 2)
            # print(my_version_dec, my_type_id_dec, pack_len_type_id, sub_packs_len_dec)
            sub_packs_tuples=parse_msg(pack_bin[7+15:7+15+sub_packs_len_dec])
            sub_packs_remaining_bin=sub_packs_tuples[0][3]
            while sub_packs_remaining_bin!='':
                sub_packs_tuple=parse_msg(sub_packs_remaining_bin)
                sub_packs_remaining_bin=sub_packs_tuple[0][3]
                sub_packs_tuples+=sub_packs_tuple
            my_remain_part = pack_bin[7+15+sub_packs_len_dec:]
            remaining_packs=nr_of_packs-1
        elif pack_len_type_id==1:  
            #Fixed count
            # print(my_version_dec, my_type_id_dec, pack_len_type_id)
            sub_packs_count_bin=pack_bin[7:7+11]
            sub_packs_count_dec=int(sub_packs_count_bin, 2)
            # print(my_version_dec, my_type_id_dec, pack_len_type_id, sub_packs_count_dec)
            sub_packs_remaining_bin=pack_bin[7+11:]
            while sub_packs_count_dec>0:
                sub_packs_tuple=parse_msg(sub_packs_remaining_bin, nr_of_packs=sub_packs_count_dec)
                sub_packs_tuples+=sub_packs_tuple
                sub_packs_remaining_bin=sub_packs_tuple[0][3]
                sub_packs_count_dec=sub_packs_tuple[0][4]
            remaining_packs=nr_of_packs-1
            my_remain_part=sub_packs_remaining_bin
        else:
            print("Pass")
            raise Exception('It should not get here!!!')
    
    my_tuple=[(my_version_dec, my_type_id_dec, my_calc_value, my_remain_part, remaining_packs)]
    return my_tuple+sub_packs_tuples

In [10]:
parsed_msg=parse_msg(hex2bin(data))
#parsed_msg

In [11]:
result=sum([subpack[0] for subpack in parsed_msg])

In [12]:
print(f"Result for part 1: {result}")

Result for part 1: 953


## Part 2

In [13]:
from math import prod

In [14]:
def parse_msg(pack_bin: str, nr_of_packs=1) -> List[Tuple[int, int, int, str, int]]:
    sub_packs_tuples=[]
    my_calc_value=0
    #Header
    my_version_bin=pack_bin[:3]
    my_version_dec=int(my_version_bin, 2)
    my_type_id_bin=pack_bin[3:6]
    my_type_id_dec=int(my_type_id_bin, 2)
    
    #Value or package
    if my_type_id_dec==4:
        # Value
        my_calc_value, my_remain_part = decode_value(pack_bin[6:])
        # print(f"Extracted value {my_calc_value}, remained {my_remain_part}")
        remaining_packs=nr_of_packs-1
    else:
        #Operator
        pack_len_type_id=int(pack_bin[6])
        # print(pack_len_type_id)
        if pack_len_type_id==0:
            #Fixed bits lengths
            sub_packs_len_bin=pack_bin[7:7+15]
            sub_packs_len_dec=int(sub_packs_len_bin, 2)
            # print(my_version_dec, my_type_id_dec, pack_len_type_id, sub_packs_len_dec)
            sub_packs_tuples=parse_msg(pack_bin[7+15:7+15+sub_packs_len_dec])
            sub_packs_remaining_bin=sub_packs_tuples[0][3]
            while sub_packs_remaining_bin!='':
                sub_packs_tuple=parse_msg(sub_packs_remaining_bin)
                sub_packs_remaining_bin=sub_packs_tuple[0][3]
                sub_packs_tuples+=sub_packs_tuple
            my_remain_part = pack_bin[7+15+sub_packs_len_dec:]
            remaining_packs=nr_of_packs-1
        elif pack_len_type_id==1:  
            #Fixed count
            # print(my_version_dec, my_type_id_dec, pack_len_type_id)
            sub_packs_count_bin=pack_bin[7:7+11]
            sub_packs_count_dec=int(sub_packs_count_bin, 2)
            # print(my_version_dec, my_type_id_dec, pack_len_type_id, sub_packs_count_dec)
            sub_packs_remaining_bin=pack_bin[7+11:]
            while sub_packs_count_dec>0:
                sub_packs_tuple=parse_msg(sub_packs_remaining_bin, nr_of_packs=sub_packs_count_dec)
                sub_packs_tuples+=sub_packs_tuple
                sub_packs_remaining_bin=sub_packs_tuple[0][3]
                sub_packs_count_dec=sub_packs_tuple[0][4]
            remaining_packs=nr_of_packs-1
            my_remain_part=sub_packs_remaining_bin
        else:
            print("Pass")
            raise Exception('It should not get here!!!')
    
    sub_pack_vals=[sub_pack[2] for sub_pack in sub_packs_tuples]
    if my_type_id_dec==0:
        my_calc_value=sum(sub_pack_vals)
    elif my_type_id_dec==1:
        my_calc_value=prod(sub_pack_vals)
    elif my_type_id_dec==2:
        my_calc_value=min(sub_pack_vals)
    elif my_type_id_dec==3:
        my_calc_value=max(sub_pack_vals)
    elif my_type_id_dec==5:
        my_calc_value=int(sub_pack_vals[0]>sub_pack_vals[1])
    elif my_type_id_dec==6:
        my_calc_value=int(sub_pack_vals[0]<sub_pack_vals[1])
    elif my_type_id_dec==7:
        my_calc_value=int(sub_pack_vals[0]==sub_pack_vals[1])
    
    # if my_type_id_dec!=4: print(my_calc_value, my_type_id_dec, sub_pack_vals)
    
    my_tuple=[(my_version_dec, my_type_id_dec, my_calc_value, my_remain_part, remaining_packs)]
    return my_tuple

In [15]:
parsed_msg=parse_msg(hex2bin(data))

In [16]:
result=parsed_msg[0][2]

In [17]:
print(f"Result for part 2: {result}")

Result for part 2: 246225449979
